## Step 1: 환경 설정 및 의존성 설치

In [3]:
# 필수 패키지 설치
!pip install -q transformers einops huggingface_hub matplotlib scikit-learn biopython torch pandas numpy

## Step 2: 모듈 임포트

In [4]:
import sys
import os
import gc
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import torch

# 현재 디렉토리를 경로에 추가 (모듈 로드용)
current_dir = Path('.').resolve()
project_root = current_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 모듈 임포트
from preparation import get_device, load_models
from sequence_generation import (
    fetch_gene_sequences, sort_genes_by_length, 
    DEFAULT_DECODING_STRATEGIES
)

print("✅ 모든 모듈 임포트 완료")

✅ 모든 모듈 임포트 완료


## Step 3: 디바이스 초기화 및 모델 로드

In [5]:
# 디바이스 감지 (CUDA / MPS / CPU)
device = get_device()
print(f"\n사용 중인 디바이스: {device}")

PyTorch Version: 2.9.1
Using device: mps

사용 중인 디바이스: mps


In [6]:
# 모델 로드
model_configs = {
    "DNABERT-2": "zhihan1996/DNABERT-2-117M",
    # "NT-v2-50m": "InstaDeepAI/nucleotide-transformer-v2-50m-multi-species",
    "NT-v2-500m": "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species",
}

models = load_models(device, model_configs=model_configs)
print(f"로드된 모델: {list(models.keys())}")


📥 Downloading DNABERT-2...


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

✅ DNABERT-2 Triton patch applied successfully.
[DNABERT-2] Loading model...


/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


[DNABERT-2] Model loaded successfully.
✅ DNABERT-2 loaded successfully.
[NT-v2-500m] Loading model...
[NT-v2-500m] Model loaded successfully.
✅ NT-v2-500m loaded successfully.

🚀 2 model(s) ready!
로드된 모델: ['DNABERT-2', 'NT-v2-500m']


## Step 4: 유전자 시퀀스 수집 및 생성

In [7]:
# 설정
NCBI_EMAIL = "your_email@example.com"
ITERATIONS = 30  # 진화 스텝 수
MASK_RATIO = 0.2  # 마스킹 비율
RESULTS_DIR = Path('results/sequences')
RESULTS_DIR.mkdir(exist_ok=True)

print(f"설정:")
print(f"  - 반복 횟수: {ITERATIONS}")
print(f"  - 마스킹 비율: {MASK_RATIO}")
print(f"  - 결과 디렉토리: {RESULTS_DIR}")

설정:
  - 반복 횟수: 30
  - 마스킹 비율: 0.2
  - 결과 디렉토리: results/sequences


In [8]:
# 유전자 시퀀스 수집
gene_selection = fetch_gene_sequences(NCBI_EMAIL)
gene_selection = sort_genes_by_length(gene_selection)

print("\n수집된 유전자:")
for gene, seq in gene_selection.items():
    print(f"  - {gene}: {len(seq)} bp")

Fetching gene sequences from NCBI...
Fetching H4C1 by UID NM_003538...
  ✅ Found and added sequence for H4C1 (Length: 402bp)
Fetching TP53 by UID NM_000546...
  ✅ Found and added sequence for TP53 (Length: 2512bp)
Fetching GAPDH by UID NM_002046...
  ✅ Found and added sequence for GAPDH (Length: 1285bp)
Fetching GAPDHP1 by UID NG_001123...
  ✅ Found and added sequence for GAPDHP1 (Length: 1098bp)
Fetching NORAD by UID NR_027451...
  ✅ Found and added sequence for NORAD (Length: 5378bp)
Fetching STAT3 by UID NM_139276...
  ✅ Found and added sequence for STAT3 (Length: 4921bp)
Fetching TTN by UID NM_001267550...
  ✅ Found and added sequence for TTN (Length: 109224bp)
Fetching HBB by UID NM_000518...
  ✅ Found and added sequence for HBB (Length: 628bp)
Fetching HOXC11 by UID NM_014212...
  ✅ Found and added sequence for HOXC11 (Length: 3261bp)
Fetching HOTAIR by UID NR_003716...
  ✅ Found and added sequence for HOTAIR (Length: 2273bp)
Fetching VEGFA by UID NM_003376...
  ✅ Found and added

In [9]:
# 시퀀스 생성 및 CSV 저장
print("\n시퀀스 생성 중...\n" + "="*70)

for model_label, model_instance in models.items():
    print(f"\n모델: {model_label}")
    
    for gene_id, original_sequence in gene_selection.items():
        print(f"  {gene_id}...", end="", flush=True)
        
        # 출력 경로
        model_name = model_label.replace("/", "-")
        model_dir = RESULTS_DIR / model_name
        model_dir.mkdir(parents=True, exist_ok=True)
        output_csv = model_dir / f"{gene_id}.csv"
        
        # 결과 저장용 데이터프레임
        results_data = {}
        
        for strategy_base_key, strategy_cfg in DEFAULT_DECODING_STRATEGIES.items():
            strategy_type = strategy_cfg["type"]
            temperatures = strategy_cfg.get("temperatures", [1.0])
            top_k = strategy_cfg.get("top_k", 50)
            
            for temp in temperatures:
                # 전략 이름
                if strategy_type == "greedy":
                    strategy_key = strategy_base_key
                else:
                    strategy_key = f"{strategy_base_key}_t{temp}"
                
                # 시퀀스 생성
                generated_sequences = model_instance.run(
                    sequence=original_sequence,
                    steps=ITERATIONS,
                    mask_ratio=MASK_RATIO,
                    strategy=strategy_type,
                    temperature=temp,
                    top_k=top_k,
                    save_all=True,
                    save_interval=1
                )
                
                # 결과 저장 (각 iteration을 column으로)
                results_data[strategy_key] = generated_sequences
                
                # 메모리 정리
                del generated_sequences
                gc.collect()
                if device == "cuda":
                    torch.cuda.empty_cache()
                elif device == "mps":
                    torch.mps.empty_cache()
        
        # DataFrame 생성 (decoding strategy를 row로)
        df = pd.DataFrame(results_data).T
        
        # 열 이름을 iteration 번호로 설정
        df.columns = [f"iteration_{i}" for i in range(df.shape[1])]
        
        # CSV 저장
        df.to_csv(output_csv)
        
        print(f" ✓ ({len(df)} strategies)")

print("\n" + "="*70)
print("✅ 시퀀스 생성 완료!")
print(f"결과 저장 위치: {RESULTS_DIR}")

# 기존 임베딩 캐시 정리 (새 시퀀스 기준으로 다시 계산 필요)
embeddings_dir = Path('results/embeddings')
if embeddings_dir.exists():
    removed = 0
    for cache_file in embeddings_dir.glob('**/*.pkl'):
        try:
            cache_file.unlink()
            removed += 1
        except Exception as e:
            print(f'  캐시 삭제 실패: {cache_file} ({e})')
    print(f'🧹 기존 임베딩 캐시 삭제 완료: {removed} files')
else:
    print('🧹 기존 임베딩 캐시 없음')



시퀀스 생성 중...

모델: DNABERT-2
  H4C1... ✓ (4 strategies)
  HBB... ✓ (4 strategies)
  GAPDHP1... ✓ (4 strategies)
  GAPDH... ✓ (4 strategies)
  TPI1... ✓ (4 strategies)
  TPI1P1... ✓ (4 strategies)
  HOTAIR...

/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 512 to 516
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 516 to 523
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 523 to 528
  warnings.warn(


 ✓ (4 strategies)
  TP53...

/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 528 to 535
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 535 to 539
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 539 to 543
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 543 to 546
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 546 to 549
  warnings.warn(
/Users/leeminjae/.cache/h

 ✓ (4 strategies)
  NEAT1...

...

 ✓ (4 strategies)
  PTENP1...

/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 822 to 828
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 828 to 835
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 835 to 840
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 840 to 842
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 842 to 844
  warnings.warn(
/Users/leeminjae/.cache/h

 ✓ (4 strategies)
  STAT3...

/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 889 to 983
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 983 to 993
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 993 to 994
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 994 to 999
  warnings.warn(
/Users/leeminjae/.cache/huggingface/modules/transformers_modules/_7bce263b15377fc15361f52cfab88f8b586abda0/bert_layers.py:433: UserWarning: Increasing alibi size from 999 to 1002
  warnings.warn(
/Users/leeminjae/.cache/

 ✓ (4 strategies)
  NORAD... ✓ (4 strategies)
  PTEN... ✓ (4 strategies)
  TTN... ✓ (4 strategies)

모델: NT-v2-500m
  H4C1... ✓ (4 strategies)
  HBB... ✓ (4 strategies)
  GAPDHP1... ✓ (4 strategies)
  GAPDH... ✓ (4 strategies)
  TPI1... ✓ (4 strategies)
  TPI1P1... ✓ (4 strategies)
  HOTAIR... ✓ (4 strategies)
  TP53... ✓ (4 strategies)
  NEAT1... ✓ (4 strategies)
  HOXC11... ✓ (4 strategies)
  VEGFA... ✓ (4 strategies)
  PTENP1... ✓ (4 strategies)
  STAT3... ✓ (4 strategies)
  NORAD... ✓ (4 strategies)
  PTEN... ✓ (4 strategies)
  TTN... ✓ (4 strategies)

✅ 시퀀스 생성 완료!
결과 저장 위치: results/sequences
🧹 기존 임베딩 캐시 삭제 완료: 4 files


## 결과 확인

In [ ]:
import pandas as pd

print("생성된 결과 확인:\n")
print("="*70)

for model_key in models.keys():
    model_name = model_key.replace("/", "-")
    model_dir = RESULTS_DIR / model_name
    
    print(f"\n{model_key}:")
    
    for gene_id in gene_selection.keys():
        csv_file = model_dir / f"{gene_id}.csv"
        if csv_file.exists():
            df = pd.read_csv(csv_file, index_col=0)
            num_strategies = len(df)
            num_iterations = len(df.columns)
            print(f"  {gene_id:10s}: {num_strategies:2d} strategies, {num_iterations:2d} iterations")
            if num_strategies > 0:
                print(f"            First strategy: {df.index[0]}")
        else:
            print(f"  {gene_id:10s}: ✗ FILE NOT FOUND")

print("\n" + "="*70)

생성된 결과 확인:


DNABERT-2:
  H4C1      :  4 strategies, 31 iterations
            First strategy: greedy
  HBB       :  4 strategies, 31 iterations
            First strategy: greedy
  GAPDHP1   :  4 strategies, 31 iterations
            First strategy: greedy
  GAPDH     :  4 strategies, 31 iterations
            First strategy: greedy
  TPI1      :  4 strategies, 31 iterations
            First strategy: greedy
  TPI1P1    :  4 strategies, 31 iterations
            First strategy: greedy
  HOTAIR    :  4 strategies, 31 iterations
            First strategy: greedy
  TP53      :  4 strategies, 31 iterations
            First strategy: greedy
  NEAT1     :  4 strategies, 31 iterations
            First strategy: greedy
  HOXC11    :  4 strategies, 31 iterations
            First strategy: greedy
  VEGFA     :  4 strategies, 31 iterations
            First strategy: greedy
  PTENP1    :  4 strategies, 31 iterations
            First strategy: greedy
  STAT3     :  4 strategies, 31 iteratio

: 